# Classification de la question en thématiques métier

## 1. Pré-traitement textuel de la base de données

### A. Préparation du fichier

Nous importons la base créée après le scraping (fichier `scraping.py`).

In [1]:
import pandas as pd

In [2]:
df_faq = pd.read_pickle('faq_centerPark.pkl')
df_faq.head()

,question,reponse,theme
0,Quels sont les jours d’arrivée ?,Il est possible d’arriver n’importe quel jour ...,Préparer mon séjour
1,Comment évaluer le confort de mon domaine et d...,Le classement par « birdies » évalue l’offre C...,Préparer mon séjour
2,Quels sont les services et activités compris d...,"En réservant votre hébergement, vous bénéficie...",Préparer mon séjour
3,Comment réserver mes activités ?,"Lors de la réservation de votre hébergement, v...",Préparer mon séjour
4,Où trouver le plan du domaine ?,"Sur la page d'accueil de notre site, cliquez s...",Préparer mon séjour


In [3]:
df_faq.shape

(54, 3)

Notre jeu de données est composé de :
    - 54 lignes
    - 3 colonnes
    - 5 thèmes différents
On pose :
    - 1 = Préparer mon séjour
    - 2 = Réserver et payer
    - 3 = Gérer ma réservation
    - 4 = Mon séjour
    - 5 = Assurances

In [4]:
dic_code_theme = {"Préparer mon séjour": 1,
                  "Réserver et payer": 2,
                  "Gérer ma réservation": 3,
                  "Mon séjour": 4,
                  "Assurances": 5}
dic_decode_theme = {val: key for key, val in dic_code_theme.items()}

In [5]:
df_faq["theme"].replace(dic_code_theme, inplace=True)

Nous allons transposer le DataFrame de manière à ce que les questions et les réponses se retrouvent dans la même colonne, tout en restant associées à leur thème d'appartenance.

In [6]:
df_question = df_faq[['question', 'theme']].copy()
df_question.rename(columns={'question': 'texte', 'theme': 'theme'}, inplace=True)
df_reponse = df_faq[['reponse', 'theme']].copy()
df_reponse.rename(columns={'reponse': 'texte', 'theme': 'theme'}, inplace=True)

In [7]:
df_theme = pd.concat([df_question, df_reponse])

Nous sauvegardons le DataFrame `df_theme`.

In [8]:
df_theme.to_pickle('df_classif_theme.pkl')

### B. Nettoyage du texte

Nous allons procéder à un nettoyage du tableau de données.

In [9]:
import os
import numpy as np
import re
from collections import Counter

# pour le nettoyage du texte
import nltk
import string
import spacy
from nltk.tokenize import word_tokenize
from nltk.stem import SnowballStemmer
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

#pour la classification
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report, confusion_matrix
from sklearn.svm import SVC

import unidecode

Nous séparons le tableau `df_theme` en un jeu d'apprentissage et un jeu de test. Nous conservons 70% des données dans le jeu d'apprentissage.

In [10]:
from sklearn.model_selection import train_test_split

df_theme = pd.read_pickle('df_classif_theme.pkl')
X_train, X_test, y_train, y_test = train_test_split(df_theme['texte'], 
                                                    df_theme['theme'],
                                                    train_size=0.7,
                                                    random_state=5)

Nous définissons un certain nombre de fonctions qui nous permettrons de nettoyer le corpus.

In [12]:
stemmer = SnowballStemmer('french')
nlp = spacy.load('fr_core_news_sm')

sw = nltk.corpus.stopwords.words('french')
sw += ['être', 'avoir']
sw.sort()

def lemmatise_text(text):
    lst_lematised = [token.lemma_ for token in nlp(text)]
    return ' '.join(lst_lematised).lower()


def stem_text(text):
    lst_stemmerised = [stemmer.stem(token) for token in word_tokenize(text)]
    return ' '.join(lst_stemmerised)


def substitute_punctuation(text):
    return ' '.join(text.replace("'", ' ').translate(str.maketrans('', '', string.punctuation)).split())


def substitute_special_char(text):
    return text.replace("«", "").replace("’", "").replace("•", "").replace("®", "")

In [13]:
X_train_clean = (X_train.apply(lemmatise_text)
                        .apply(stem_text)
                        .apply(substitute_punctuation)
                        .apply(substitute_special_char)
                )

X_test_clean = (X_test.apply(lemmatise_text)
                      .apply(stem_text)
                      .apply(substitute_punctuation)
                      .apply(substitute_special_char)
               )

## 2. Test de différents modèles

Nous allons maintenant procéder aux tests de différents classifieurs. Pour ce faire, nous allons définir plusieurs vectoriseurs.

### A. Les différents vectoriseurs

In [14]:
#Vectoriseur binaire

bin_count = CountVectorizer(binary=True)
bin_count.fit(X_train_clean)
X_train_clean_vectorized_bin = bin_count.transform(X_train_clean)
X_test_clean_vectorized_bin = bin_count.transform(X_test_clean)

In [15]:
#vectoriseur numérique discret

vect_count = CountVectorizer(binary=False)
vect_count.fit(X_train_clean)
X_train_clean_vectorized_count = vect_count.transform(X_train_clean)
X_test_clean_vectorized_count = vect_count.transform(X_test_clean)

In [16]:
#vectoriseur numérique continu : TF-IDF

vect_tfidf = TfidfVectorizer(stop_words=sw)
vect_tfidf.fit(X_train_clean)
X_train_clean_vectorized_tfidf = vect_tfidf.transform(X_train_clean)
X_test_clean_vectorized_tfidf = vect_tfidf.transform(X_test_clean) 

### B. Les différents modèles

Nous entraînerons des modèles de classification appartenant à quelques familles d'algorithmes d'apprentissage automatique classique. L'objectif est de comparer non seulement les performances des différentes méthodes entre elles, mais aussi la performance d'une même méthode sur des représentations différentes du texte.

#### B.1 DummyClassifier

In [17]:
from sklearn.dummy import DummyClassifier

In [18]:
random_uniform = DummyClassifier(strategy='uniform').fit(X_train_clean_vectorized_count, y_train)
predictions_valid = random_uniform.predict(X_test_clean_vectorized_count)
accuracy_score(y_test, predictions_valid)

0.24242424242424243

In [19]:
random_uniform = DummyClassifier(strategy='uniform').fit(X_train_clean_vectorized_bin, y_train)
predictions_valid = random_uniform.predict(X_test_clean_vectorized_bin)
accuracy_score(y_test, predictions_valid)

0.15151515151515152

In [20]:
random_uniform = DummyClassifier(strategy='uniform').fit(X_train_clean_vectorized_tfidf, y_train)
predictions_valid = random_uniform.predict(X_test_clean_vectorized_tfidf)
accuracy_score(y_test, predictions_valid)

0.12121212121212122

#### B.2 Classifieur naïf bayesien

In [21]:
from sklearn.naive_bayes import MultinomialNB

In [22]:
model_nb = MultinomialNB().fit(X_train_clean_vectorized_count, y_train)
predictions_valid = model_nb.predict(X_test_clean_vectorized_count)
accuracy_score(y_test, predictions_valid)

0.5757575757575758

In [23]:
model_nb = MultinomialNB().fit(X_train_clean_vectorized_bin, y_train)
predictions_valid = model_nb.predict(X_test_clean_vectorized_bin)
accuracy_score(y_test, predictions_valid)

0.6060606060606061

In [24]:
model_nb = MultinomialNB().fit(X_train_clean_vectorized_tfidf, y_train)
predictions_valid = model_nb.predict(X_test_clean_vectorized_tfidf)
accuracy_score(y_test, predictions_valid)

0.5454545454545454

#### B.3 Complement NB

In [25]:
from sklearn.naive_bayes import ComplementNB

In [26]:
model_comp = ComplementNB().fit(X_train_clean_vectorized_count, y_train)
predictions_valid = model_comp.predict(X_test_clean_vectorized_count)
accuracy_score(y_test, predictions_valid)

0.5454545454545454

In [27]:
model_comp = ComplementNB().fit(X_train_clean_vectorized_bin, y_train)
predictions_valid = model_comp.predict(X_test_clean_vectorized_bin)
accuracy_score(y_test, predictions_valid)

0.6060606060606061

In [28]:
model_comp = ComplementNB().fit(X_train_clean_vectorized_tfidf, y_train)
predictions_valid = model_comp.predict(X_test_clean_vectorized_tfidf)
accuracy_score(y_test, predictions_valid)

0.6666666666666666

#### B.4 BernoulliNB

In [29]:
from sklearn.naive_bayes import BernoulliNB

In [30]:
model_bern = BernoulliNB().fit(X_train_clean_vectorized_count, y_train)
predictions_valid = model_bern.predict(X_test_clean_vectorized_count)
accuracy_score(y_test, predictions_valid)

0.36363636363636365

In [31]:
model_bern = BernoulliNB().fit(X_train_clean_vectorized_bin, y_train)
predictions_valid = model_bern.predict(X_test_clean_vectorized_bin)
accuracy_score(y_test, predictions_valid)

0.36363636363636365

In [32]:
model_bern = BernoulliNB().fit(X_train_clean_vectorized_tfidf, y_train)
predictions_valid = model_bern.predict(X_test_clean_vectorized_tfidf)
accuracy_score(y_test, predictions_valid)

0.36363636363636365

#### B.5 KNeighborsClassifier

In [33]:
from sklearn.neighbors import KNeighborsClassifier

In [34]:
model_knn = KNeighborsClassifier(4).fit(X_train_clean_vectorized_count, y_train)
predictions_valid = model_knn.predict(X_test_clean_vectorized_count)
accuracy_score(y_test, predictions_valid)

0.2727272727272727

In [35]:
model_knn = KNeighborsClassifier(4).fit(X_train_clean_vectorized_bin, y_train)
predictions_valid = model_knn.predict(X_test_clean_vectorized_bin)
accuracy_score(y_test, predictions_valid)

0.30303030303030304

In [36]:
model_knn = KNeighborsClassifier(4).fit(X_train_clean_vectorized_tfidf, y_train)
predictions_valid = model_knn.predict(X_test_clean_vectorized_tfidf)
accuracy_score(y_test, predictions_valid)

0.6363636363636364

#### B.7 SVM

In [37]:
from sklearn.svm import SVC

In [38]:
model_svm = SVC(kernel='linear', C=0.9).fit(X_train_clean_vectorized_count, y_train)
predictions_valid = model_svm.predict(X_test_clean_vectorized_count)
accuracy_score(y_test, predictions_valid)

0.3939393939393939

In [39]:
model_svm = SVC(kernel='linear', C=0.9).fit(X_train_clean_vectorized_bin, y_train)
predictions_valid = model_svm.predict(X_test_clean_vectorized_bin)
accuracy_score(y_test, predictions_valid)

0.3939393939393939

In [40]:
model_svm = SVC(kernel='linear', C=0.9).fit(X_train_clean_vectorized_tfidf, y_train)
predictions_valid = model_svm.predict(X_test_clean_vectorized_tfidf)
accuracy_score(y_test, predictions_valid)

0.5757575757575758

#### B.8 Réseaux de neurones de convolution

In [41]:
from keras.models import Sequential
from keras.layers import LSTM, Embedding, Dense, SpatialDropout1D, MaxPooling1D, Conv1D, Flatten, MaxPooling2D, Conv2D

Using TensorFlow backend.


In [42]:
XX_train = X_train_clean_vectorized_tfidf.toarray().reshape(X_train_clean_vectorized_tfidf.shape[0],1,
                                                X_train_clean_vectorized_tfidf.shape[1])
XX_test = X_test_clean_vectorized_tfidf.toarray().reshape(X_test_clean_vectorized_tfidf.shape[0],1,
                                                X_test_clean_vectorized_tfidf.shape[1])
YY_train = pd.get_dummies(y_train)
YY_test = pd.get_dummies(y_test)

In [43]:
n_timesteps = 1
n_features = XX_train.shape[2]

model = Sequential()
model.add(Conv1D(filters=32, kernel_size=1, activation='relu', input_shape=(n_timesteps, n_features)))
model.add(MaxPooling1D(pool_size=1))
model.add(Flatten())
model.add(Dense(16, activation='relu'))
model.add(Dense(units=YY_train.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer="adam", metrics=['accuracy'])

epochs = 150
batch_size = 32
history = model.fit(XX_train, YY_train, epochs=epochs, batch_size=batch_size, validation_split=0.4)

Train on 45 samples, validate on 30 samples
Epoch 1/150
45/45 [==============================] - ETA: 0s - loss: 1.6041 - accuracy: 0.18 - 0s 8ms/step - loss: 1.5982 - accuracy: 0.2444 - val_loss: 1.5939 - val_accuracy: 0.2667
Epoch 2/150
45/45 [==============================] - ETA: 0s - loss: 1.5916 - accuracy: 0.28 - 0s 467us/step - loss: 1.5888 - accuracy: 0.2667 - val_loss: 1.5904 - val_accuracy: 0.2333
Epoch 3/150
45/45 [==============================] - ETA: 0s - loss: 1.5857 - accuracy: 0.25 - 0s 444us/step - loss: 1.5810 - accuracy: 0.2889 - val_loss: 1.5864 - val_accuracy: 0.2333
Epoch 4/150
45/45 [==============================] - ETA: 0s - loss: 1.5715 - accuracy: 0.28 - 0s 400us/step - loss: 1.5737 - accuracy: 0.2889 - val_loss: 1.5821 - val_accuracy: 0.2333
Epoch 5/150
45/45 [==============================] - ETA: 0s - loss: 1.5559 - accuracy: 0.37 - 0s 467us/step - loss: 1.5661 - accuracy: 0.3111 - val_loss: 1.5777 - val_accuracy: 0.2667
Epoch 6/150
45/45 [==============

Epoch 45/150
45/45 [==============================] - ETA: 0s - loss: 0.8718 - accuracy: 0.68 - 0s 378us/step - loss: 0.8633 - accuracy: 0.7556 - val_loss: 1.2418 - val_accuracy: 0.4667
Epoch 46/150
45/45 [==============================] - ETA: 0s - loss: 0.8370 - accuracy: 0.75 - 0s 378us/step - loss: 0.8414 - accuracy: 0.7778 - val_loss: 1.2328 - val_accuracy: 0.5000
Epoch 47/150
45/45 [==============================] - ETA: 0s - loss: 0.8407 - accuracy: 0.81 - 0s 556us/step - loss: 0.8192 - accuracy: 0.8444 - val_loss: 1.2248 - val_accuracy: 0.5000
Epoch 48/150
45/45 [==============================] - ETA: 0s - loss: 0.8429 - accuracy: 0.81 - 0s 422us/step - loss: 0.7982 - accuracy: 0.8444 - val_loss: 1.2168 - val_accuracy: 0.5000
Epoch 49/150
45/45 [==============================] - ETA: 0s - loss: 0.7468 - accuracy: 0.90 - 0s 422us/step - loss: 0.7774 - accuracy: 0.8667 - val_loss: 1.2081 - val_accuracy: 0.5000
Epoch 50/150
45/45 [==============================] - ETA: 0s - loss: 

Epoch 133/150
45/45 [==============================] - ETA: 0s - loss: 0.0984 - accuracy: 0.96 - 0s 378us/step - loss: 0.0920 - accuracy: 0.9778 - val_loss: 1.0122 - val_accuracy: 0.5333
Epoch 134/150
45/45 [==============================] - ETA: 0s - loss: 0.1137 - accuracy: 0.96 - 0s 378us/step - loss: 0.0907 - accuracy: 0.9778 - val_loss: 1.0128 - val_accuracy: 0.5333
Epoch 135/150
45/45 [==============================] - ETA: 0s - loss: 0.0867 - accuracy: 0.96 - 0s 400us/step - loss: 0.0894 - accuracy: 0.9778 - val_loss: 1.0136 - val_accuracy: 0.5333
Epoch 136/150
45/45 [==============================] - ETA: 0s - loss: 0.0678 - accuracy: 1.00 - 0s 422us/step - loss: 0.0884 - accuracy: 0.9778 - val_loss: 1.0140 - val_accuracy: 0.5333
Epoch 137/150
45/45 [==============================] - ETA: 0s - loss: 0.1036 - accuracy: 0.96 - 0s 400us/step - loss: 0.0866 - accuracy: 0.9778 - val_loss: 1.0139 - val_accuracy: 0.5333
Epoch 138/150
45/45 [==============================] - ETA: 0s - 

In [44]:
pred_proba=model.predict(XX_test)
idx = np.argmax(pred_proba, axis=-1)
YY_pred = np.zeros( pred_proba.shape )
YY_pred[ np.arange(YY_pred.shape[0]), idx] = 1
accuracy_score(YY_test, YY_pred)

0.6666666666666666

## 3. Choix et sauvegarde

Bien que nous n'ayons pas laissé la trace de nos recherches, nous avons cherché, pour chaque modèle, à ajuster au mieux leurs paramètres. Après ces différents tests, nous sauvegardons le modèle associé au classifieur qui nous fournit la meilleure accuracy. Ainsi, nous choisissons le réseau de neuronnes à convolution et le classifieur TF-IDF.

In [45]:
from joblib import dump
dump(model, 'model_classif_theme.joblib')

['model_classif_theme.joblib']

In [47]:
dump(vect_tfidf, 'vectorizer_classif_theme.joblib')

['vectorizer_classif_theme.joblib']

## 4. Evaluation quantitative

Afin d'évaluer de manière quantitative le modèle sélectionné, nous avons créé un jeu de questions - thèmes dont les thèmes correspondent à ceux de la FAQ mais dont les questions ont été imaginées pas nos soins. L'objectif étant de comparer la prédiction du modèle choisi au thème attendu.

In [72]:
df_test_quanti = pd.read_csv('jeu_test_theme.csv', ";")
df_test_quanti.head()

,question,theme
0,Quels sont les jours d’arrivée ?,Préparer mon séjour
1,Comment évaluer le confort de mon domaine et d...,Préparer mon séjour
2,Quels sont les services et activités compris d...,Préparer mon séjour
3,Comment réserver mes activités ?,Préparer mon séjour
4,Où trouver le plan du domaine ?,Préparer mon séjour


Nous appliquons le même traitement que précédemment à ce jeu de données. Nous codons les thèmes puis nous séparons la variable d'intérêt de la variable explicative.

In [73]:
df_test_quanti["theme"].replace(dic_code_theme, inplace=True)
X_test_quanti = df_test_quanti.question
y_test_quanti = df_test_quanti.theme

In [74]:
X_test_quanti = (X_test_quanti.apply(lemmatise_text)
                              .apply(stem_text)
                              .apply(substitute_punctuation)
                              .apply(substitute_special_char)
               )

In [75]:
X_test_quanti_vectorized_tfidf = vect_tfidf.transform(X_test_quanti)

In [76]:
XX_test_quanti = X_test_quanti_vectorized_tfidf.toarray().reshape(X_test_quanti_vectorized_tfidf.shape[0],1,
                                                X_test_quanti_vectorized_tfidf.shape[1])
YY_test_quanti = pd.get_dummies(y_test_quanti)

In [77]:
pred_proba=model.predict(XX_test_quanti)
idx = np.argmax(pred_proba, axis=-1)
YY_pred_quanti = np.zeros( pred_proba.shape )
YY_pred_quanti[ np.arange(YY_pred_quanti.shape[0]), idx] = 1

In [81]:
accuracy_score(YY_test_quanti, YY_pred_quanti)

0.6852367688022284

In [82]:
f1_score(YY_test_quanti, YY_pred_quanti, average='weighted')

0.6698296185567882

In [87]:
precision_score(YY_test_quanti, YY_pred_quanti, average='weighted')

0.6977658241409519

In [88]:
recall_score(YY_test_quanti, YY_pred_quanti, average='weighted')

0.6852367688022284

Les résultats obtenus sont satisfaisants puisque l'introduction de nouvelles questions ne perturbent pas le modèle. Le taux de nouvelles questions bien classées reste proche des 70%.